<a href="https://colab.research.google.com/github/hui-001/DepthMaster/blob/main/Depth%20Anything/Predicting_depth_in_an_image_with_Depth_Anything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-milryeid
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-milryeid
  Resolved https://github.com/huggingface/transformers.git to commit 626666c444208557041ef7edcda6b9e78eddfdee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Load image

We'll perform inference on the familiar cats image.

In [60]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import torch
from PIL import Image
import os

In [61]:
image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf")

In [62]:
image_folder = "/content/D:/blender/results/001-color"
output_folder = "/content/D:/blender/results/001-pre"

In [63]:
os.makedirs(output_folder, exist_ok=True)

In [64]:
for filename in os.listdir(image_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):  # 只处理常见的图片格式
        # 构建图片的完整路径
        image_path = os.path.join(image_folder, filename)

        # 打开图片
        image = Image.open(image_path)

        # 准备图像输入
        inputs = image_processor(images=image, return_tensors="pt")

In [65]:
with torch.no_grad():
    outputs = model(**inputs)

In [66]:
post_processed_output = image_processor.post_process_depth_estimation(
            outputs,
            target_sizes=[(image.height, image.width)],)

In [67]:
predicted_depth = post_processed_output[0]["predicted_depth"]

In [68]:
depth = (predicted_depth - predicted_depth.min()) / (predicted_depth.max() - predicted_depth.min())
depth = depth.detach().cpu().numpy() * 255
depth = depth.astype("uint8")

In [69]:
depth_image = Image.fromarray(depth)

In [70]:
output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_depth.png")

In [71]:
depth_image.save(output_path)